In [1]:

cd _scripts/

/Users/ben/prj/fab/_scripts


In [12]:


%load_ext autoreload
%autoreload


import fill_untappd
import yaml, json
import os
from pprint import pprint
from airtable import Airtable
import pandas as pd

fill_untappd.run()

import pandas as pd
import sqlite3
conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins;", conn)
jsons  = df.json.apply(lambda j: json.loads(j))

venues = jsons.apply(
    lambda x: 
    pd.concat([
        pd.Series(x["venue"]),
        pd.Series(x["user"])[["user_name"]],
        pd.Series(x["beer"]),
        pd.Series(x)[["checkin_id","created_at","rating_score"]]
        ])
        )

venues_info = pd.concat([venues, 
pd.concat([
    venues['contact'].apply(lambda x: pd.Series(x)),
    venues['location'].apply(lambda x: pd.Series(x)),
], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins")

venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts],axis=1))

venues_annotated.to_csv("../_data/venues.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 845704440}
{'count': 0, 'items': []}


In [6]:
fill_untappd.run()


{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 845704440}
{'count': 0, 'items': []}


In [9]:
import sqlite3
conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins limit 100;", conn)

,cid,timestamp,beer,venue,json
0,838537272,1575735296,Squid Pro Quo,Dorchester Brewing Company,"{""checkin_id"": 838537272, ""created_at"": ""Sat, ..."
1,840159586,1576099768,Squid Pro Quo,Dorchester Brewing Company,"{""checkin_id"": 840159586, ""created_at"": ""Wed, ..."
2,843595254,1576871652,Squid Pro Quo,,"{""checkin_id"": 843595254, ""created_at"": ""Fri, ..."
3,843598886,1576871947,Squid Pro Quo,Dorchester Brewing Company,"{""checkin_id"": 843598886, ""created_at"": ""Fri, ..."
4,844777755,1577033160,Balloon Factory,Flatbread Company at Sacco's Bowl Haven,"{""checkin_id"": 844777755, ""created_at"": ""Sun, ..."
5,844837932,1577039748,Squid Pro Quo,High Tor State Park,"{""checkin_id"": 844837932, ""created_at"": ""Sun, ..."
6,845367612,1577132123,Balloon Factory,,"{""checkin_id"": 845367612, ""created_at"": ""Mon, ..."
7,845370113,1577132354,Squid Pro Quo,,"{""checkin_id"": 845370113, ""created_at"": ""Mon, ..."
8,845401472,1577135193,Squid Pro Quo,,"{""checkin_id"": 845401472, ""created_at"": ""Mon, ..."
9,845483235,1577142903,Squid Pro Quo,,"{""checkin_id"": 845483235, ""created_at"": ""Mon, ..."
